In [1]:
import numpy as np
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km

    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dLat/2)**2 + cos(lat1) * cos(lat2) * sin(dLon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))

    return R * c

def generate_centers(radius):
    # Approximate bounding box coordinates for Germany
    min_lat, max_lat = 47.270, 55.058
    min_lon, max_lon = 5.866, 15.042

    km_lat = 111  # 1 degree of latitude is approximately 111 km
    km_lon = 111 * cos(np.mean([min_lat, max_lat]))  # Adjust longitude length according to latitude

    step_lat = radius / km_lat
    step_lon = radius / km_lon

    centers = []

    lat = min_lat
    while lat < max_lat:
        lon = min_lon
        while lon < max_lon:
            centers.append((lat, lon))
            lon += step_lon
        lat += step_lat

    return centers

if __name__ == "__main__":
    radius = 50  # 50 km radius
    centers = generate_centers(radius)

    for center in centers:
        print(f"Latitude: {center[0]:.6f}, Longitude: {center[1]:.6f}")


Latitude: 47.270000, Longitude: 5.866000
Latitude: 47.270000, Longitude: 6.589301
Latitude: 47.270000, Longitude: 7.312602
Latitude: 47.270000, Longitude: 8.035903
Latitude: 47.270000, Longitude: 8.759203
Latitude: 47.270000, Longitude: 9.482504
Latitude: 47.270000, Longitude: 10.205805
Latitude: 47.270000, Longitude: 10.929106
Latitude: 47.270000, Longitude: 11.652407
Latitude: 47.270000, Longitude: 12.375708
Latitude: 47.270000, Longitude: 13.099008
Latitude: 47.270000, Longitude: 13.822309
Latitude: 47.270000, Longitude: 14.545610
Latitude: 47.720450, Longitude: 5.866000
Latitude: 47.720450, Longitude: 6.589301
Latitude: 47.720450, Longitude: 7.312602
Latitude: 47.720450, Longitude: 8.035903
Latitude: 47.720450, Longitude: 8.759203
Latitude: 47.720450, Longitude: 9.482504
Latitude: 47.720450, Longitude: 10.205805
Latitude: 47.720450, Longitude: 10.929106
Latitude: 47.720450, Longitude: 11.652407
Latitude: 47.720450, Longitude: 12.375708
Latitude: 47.720450, Longitude: 13.099008
Lati

In [1]:
import folium


m = folium.Map(location=[45.5236, -122.6750])



In [2]:
m

In [3]:
import numpy as np
import folium
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km

    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dLat/2)**2 + cos(lat1) * cos(lat2) * sin(dLon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))

    return R * c

def generate_centers(radius):
    min_lat, max_lat = 47.270, 55.058
    min_lon, max_lon = 5.866, 15.042

    km_lat = 111
    km_lon = 111 * cos(np.mean([min_lat, max_lat]))

    step_lat = radius / km_lat
    step_lon = radius / km_lon

    centers = []

    lat = min_lat
    while lat < max_lat:
        lon = min_lon
        while lon < max_lon:
            centers.append((lat, lon))
            lon += step_lon
        lat += step_lat

    return centers

def create_map(centers, radius):
    germany_center = (np.mean([47.270, 55.058]), np.mean([5.866, 15.042]))
    map_germany = folium.Map(location=germany_center, zoom_start=6)

    for center in centers:
        folium.Circle(
            location=center,
            radius=radius * 1000,  # Convert radius to meters
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.25
        ).add_to(map_germany)

    return map_germany

if __name__ == "__main__":
    radius = 50
    centers = generate_centers(radius)
    map_germany = create_map(centers, radius)
    map_germany.save("germany_circles.html")


In [10]:
import folium
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, Polygon
from pyproj import Transformer

def create_hexagonal_grid_centers(country_boundary, radius):
    bbox = country_boundary.bounds
    min_x, min_y, max_x, max_y = bbox

    x_interval = radius * 3 / 2
    y_interval = np.sqrt(3) * radius

    grid_centers = []

    x_offset = 0
    y = min_y
    while y < max_y:
        x = min_x + x_offset
        while x < max_x:
            point = Point(x, y)
            if country_boundary.contains(point):
                grid_centers.append((x, y))
            x += x_interval
        x_offset = x_interval / 2 if x_offset == 0 else 0
        y += y_interval

    return grid_centers

def create_map(centers, radius):
    germany_center = (np.mean([47.270, 55.058]), np.mean([5.866, 15.042]))
    map_germany = folium.Map(location=germany_center, zoom_start=6)

    for center in centers:
        folium.Circle(
            location=(center[1], center[0]),  # Swap the order of latitude and longitude
            radius=radius * 1000,  # Convert radius to meters
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.15
        ).add_to(map_germany)

    return map_germany

if __name__ == "__main__":
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    germany_geo = world[world['name'] == 'Germany'].geometry.iloc[0]

    # Create a GeoDataFrame with Germany's geometry
    germany_gdf = gpd.GeoDataFrame({'geometry': [germany_geo]}, crs="EPSG:4326")

    # Project to UTM Zone 32N (EPSG:32632) which covers Germany
    germany_utm = germany_gdf.to_crs("EPSG:32632").geometry.iloc[0]
    radius = 50

    centers_utm = create_hexagonal_grid_centers(germany_utm, radius * 1000) # Convert radius to meters
    transformer = Transformer.from_crs("EPSG:32632", "EPSG:4326", always_xy=True)
    centers_geo = [transformer.transform(x, y) for x, y in centers_utm]

    map_germany = create_map(centers_geo, radius)
    map_germany.save("germany_circles.html")
